## Setup

In [6]:
from dotenv import load_dotenv

load_dotenv()
llm_config = {"model": "gpt-4o-mini"}

In [7]:
task = "Write an essay about the concept fo agents in LLM, search for 3 papers in arxiv and explain concepts associated with it"

## Build groupchat

In [8]:
import autogen

In [9]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
                   "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
                   "what information is needed to complete the task. "
                   "Please note that the information will all be retrieved using"
                   " Python code. Please only suggest information that can be "
                   "retrieved using Python code. "
                   "After each step is done by others, check the progress and "
                   "instruct the remaining steps. If a step fails, try to "
                   "workaround",
    description="Given a task, determine what "
                "information is needed to complete the task. "
                "After each step is done by others, check the progress and "
                "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
                "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
                   "Please write blogs in markdown format (with relevant titles)"
                   " and put the content in pseudo ```md``` code block. "
                   "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
                "write blogs based on the code execution results and take "
                "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
                "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

critic = autogen.ConversableAgent(
    name="Critic",
    llm_config=llm_config,
    system_message="Critic."
                   "Please critic the essay written by the writer and suggest improvements."
                   "The essay must be at PHd level",
    description="Critic."
                "Critique the essay written by the writer and suggest improvements."
)

In [10]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner, critic],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer, critic],
        critic: [writer, planner],
    },
    speaker_transitions_type="allowed",
)

In [11]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = await user_proxy.a_initiate_chat(  # noqa: F704
    manager,
    message=task,
)

Admin (to chat_manager):

Write an essay about the concept fo agents in LLM, search for 3 papers in arxiv and explain concepts associated with it

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To complete the task of writing an essay about the concept of agents in Large Language Models (LLMs) and explaining related concepts from three papers found on arXiv, we will follow these steps:

1. **Search for Papers**: Use Python code to query the arXiv API for recent papers related to LLMs and the concept of agents.
2. **Extract Relevant Information**: Retrieve titles, abstracts, and possibly other relevant details from the selected papers to gain insights into the concepts discussed.
3. **Summarize Findings**: Summarize the concepts and findings from the papers and structure them into an essay format.

### Step 1: Search for Papers
We need to gather information on how to

    # Nvidia's Stock Price Performance: A Look at the Past Month

As of April 23, 2024, Nvidia Corporation (NASDAQ: NVDA) continues to captivate investors and analysts alike with its ever-evolving stock price. Over the past month, Nvidia's stock has showcased both impressive surges and slight pullbacks, driven by market trends, company announcements, and industry dynamics.

## A Month in Review: Key Movements

![Nvidia Stock Price](nvda_stock_price_apr_2024.png)

### Early April Surge
At the beginning of the month, Nvidia's stock price experienced a noticeable uptick. This rise can be attributed to several factors:

- **Strong Earnings Report**: Nvidia's quarterly earnings report released at the end of March exceeded market expectations. The company reported significant growth in revenue and profitability, bolstered by its data center and gaming divisions.
  
- **AI and Data Center Growth**: The increasing demand for artificial intelligence (AI) and data center solutions has positioned Nvidia as a key player in the tech industry. Announcements of new AI partnerships and data center expansions contributed to investor optimism.

- **Analyst Upgrades**: Several high-profile analysts upgraded their price targets for Nvidia, citing the company’s robust product pipeline and leadership in AI technology. These upgrades spurred positive sentiment in the market.

### Mid-April Volatility
As we moved into mid-April, Nvidia's stock price exhibited some volatility:

- **Market Corrections**: Broader market corrections and profit-taking activities led to fluctuations in Nvidia’s stock. Despite the company’s strong fundamentals, external market factors often play a role in short-term price movements.

- **Geopolitical Concerns**: Ongoing geopolitical tensions, particularly concerning semiconductor supply chains, created uncertainty. While Nvidia is a key innovator, its reliance on global supply chains means it’s not immune to these macroeconomic factors.

### Late April Resilience
In the latter half of April, Nvidia's stock showed remarkable resilience, recovering from mid-month lows:

- **Product Announcements**: Nvidia’s unveiling of new product lines, including the next generation of GPUs and advancements in AI technology, revived investor interest. The anticipation of future growth opportunities led to renewed buying pressure.

- **Technological Milestones**: News of Nvidia’s breakthroughs in quantum computing and collaboration with leading tech firms strengthened market confidence.

## Conclusion: A Promising Outlook
Despite the inherent short-term volatility, the past month has demonstrated Nvidia’s strong position in the tech industry and its potential for continued growth. Investors appear hopeful, given the company’s innovative strides and pivotal role in high-growth sectors like AI and gaming.

As Nvidia continues to navigate the complexities of the global market, the coming months will undoubtedly be exciting for shareholders and market observers alike. Keeping an eye on upcoming earnings reports, product launches, and broader market conditions will be crucial for those looking to understand Nvidia's stock trajectory.

Stay tuned for more updates on Nvidia and other key players in the tech industry.

In [12]:
groupchat_result.cost

{'usage_including_cached_inference': {'total_cost': 0.0033824999999999992,
  'gpt-4o-mini-2024-07-18': {'cost': 0.0033824999999999992,
   'prompt_tokens': 18650,
   'completion_tokens': 975,
   'total_tokens': 19625}},
 'usage_excluding_cached_inference': {'total_cost': 0.0033824999999999992,
  'gpt-4o-mini-2024-07-18': {'cost': 0.0033824999999999992,
   'prompt_tokens': 18650,
   'completion_tokens': 975,
   'total_tokens': 19625}}}

In [13]:
groupchat_result.chat_history

[{'content': 'Write an essay about the concept fo agents in LLM, search for 3 papers in arxiv and explain concepts associated with it',
  'role': 'assistant'},
 {'content': "To complete the task of writing an essay about the concept of agents in Large Language Models (LLMs) and explaining related concepts from three papers found on arXiv, we will follow these steps:\n\n1. **Search for Papers**: Use Python code to query the arXiv API for recent papers related to LLMs and the concept of agents.\n2. **Extract Relevant Information**: Retrieve titles, abstracts, and possibly other relevant details from the selected papers to gain insights into the concepts discussed.\n3. **Summarize Findings**: Summarize the concepts and findings from the papers and structure them into an essay format.\n\n### Step 1: Search for Papers\nWe need to gather information on how to search for papers related to agents in LLMs. For this, we can use the arXiv API. \n\nHere’s the information that is needed:\n- Keyword

In [14]:
engineer.chat_messages

defaultdict(list,
            {<autogen.agentchat.groupchat.GroupChatManager at 0x12bece690>: [{'content': 'Write an essay about the concept fo agents in LLM, search for 3 papers in arxiv and explain concepts associated with it',
               'name': 'Admin',
               'role': 'user'},
              {'content': "To complete the task of writing an essay about the concept of agents in Large Language Models (LLMs) and explaining related concepts from three papers found on arXiv, we will follow these steps:\n\n1. **Search for Papers**: Use Python code to query the arXiv API for recent papers related to LLMs and the concept of agents.\n2. **Extract Relevant Information**: Retrieve titles, abstracts, and possibly other relevant details from the selected papers to gain insights into the concepts discussed.\n3. **Summarize Findings**: Summarize the concepts and findings from the papers and structure them into an essay format.\n\n### Step 1: Search for Papers\nWe need to gather informati

Advancing AI: Insights into Agents in Large Language Models\n\n## Introduction\nThe advent of Large Language Models (LLMs) has ushered in a new era in artificial intelligence, particularly in the realm of task-oriented agents. These **agents** use LLMs to not only understand and generate human-like responses across varied domains but also to perform complex tasks autonomously. Agents, in this context, refer to AI systems that can perceive their environment, process information, and take actions to achieve specific goals. Understanding how agents function within this framework is crucial for improving their overall effectiveness and adaptability. This essay explores recent advancements in the study of agents within LLMs, drawing insights from three significant papers published on arXiv, which underscore the importance of objective evaluation, feedback mechanisms, and real-world applications.\n\n## Paper Summaries and Insights\n\n### Towards Objectively Benchmarking Social Intelligence for Language Agents at Action Level\n**Authors:** Chenxu Wang, Bin Dai, Huaping Liu, Baoyuan Wang  \n**[Link to paper](http://arxiv.org/abs/2404.05337v1)**  \nThis paper introduces the **Social Tasks in Sandbox Simulation (STSS)** benchmark, aimed at evaluating language agents' social intelligence beyond traditional metrics that focus solely on language outputs or rely on subjective assessments. The authors argue for an objective action-level assessment, focusing on how well agents achieve goals in multi-agent simulations. This objective benchmark not only provides a clearer picture of an agent's capabilities but also differentiates between the intricacies of various architectures. This work underscores the importance of a realistic environment in testing the social intelligence of LLM-based agents.\n\n### Retroformer: Retrospective Large Language Agents with Policy Gradient Optimization\n**Authors:** Weiran Yao, Shelby Heinecke, Juan Carlos Niebles, Zhiwei Liu, and others  \n**[Link to paper](http://arxiv.org/abs/2308.02151v3)**  \nIn this paper, the authors present a pioneering framework for optimizing LLMs as agents through **policy gradient optimization**. The proposed method, dubbed **Retroformer**, enhances the performance of language agents by refining their operational prompts based on feedback from their environment. Experimental results highlight a significant improvement in agent performance on various tasks, illustrating the effectiveness of learning from iterative feedback loops rather than static training paradigms. This approach marks a crucial step toward developing more adaptive and efficient agent architectures.\n\n### Large Language Model-Enabled Multi-Agent Manufacturing Systems\n**Authors:** Jonghan Lim, Birgit Vogel-Heuser, Ilya Kovalenko  \n**[Link to paper](http://arxiv.org/abs/2406.01893v2)**  \nThis research examines the integration of LLMs in **multi-agent manufacturing systems** to boost operational adaptability. By allowing agents to interpret and execute instructions in natural language, this framework enhances the overall efficiency and responsiveness of manufacturing processes. The authors demonstrate that LLMs can effectively bridge the gap between complex human instructions and machine understanding, paving the way for a more flexible and intelligent operational landscape. Through concrete case studies, the findings reveal the potential of employing language models to enhance agent collaboration and decision-making in various contexts.\n\n## Conceptual Discussion\nThe convergence of LLMs and agent architectures presents a dynamic frontier for AI research. The papers discussed highlight a common trend: the transition from simple task-response systems to more nuanced models capable of understanding context, reasoning, and learning from experience. Objective benchmarks, like the STSS, play a pivotal role in evaluating agent capabilities, providing transparency that can drive improvements. Likewise, the integration of feedback mechanisms, showcased by the Retroformer, illustrates the necessity for agents to adapt continuously. Finally, applying LLMs in practical domains, such as manufacturing, signifies a trend toward utilizing these models in real-world scenarios, reflecting their potential for industry and autonomous systems.\n\n## Conclusion\nAs research in LLM-based agents continues to evolve, it becomes increasingly evident that the integration of advanced evaluation metrics, adaptive learning techniques, and real-world applications is vital for unlocking their full potential. Moreover, as we improve agent capabilities, we must also consider the ethical implications of these advancements, particularly concerning the autonomy and decision-making processes of AI systems. The insights derived from recent studies not only highlight the importance of rigorous benchmarks and optimization strategies but also pave the way for future developments in AI agent systems. By focusing on these areas, researchers can foster the growth of capable, efficient, and intelligent agents that can navigate complex environments and tasks effectively, setting the stage for innovative applications in everyday life. \n\nWe invite our readers to share their thoughts on these advancements and engage in discussions about the future of agents in AI, as collaborative exploration will undoubtedly drive the field forward.\n\n## References\n1. **Towards Objectively Benchmarking Social Intelligence for Language Agents at Action Level** - [arXiv:2404.05337v1](http://arxiv.org/abs/2404.05337v1)\n2. **Retroformer: Retrospective Large Language Agents with Policy Gradient Optimization** - [arXiv:2308.02151v3](http://arxiv.org/abs/2308.02151v3)\n3. **Large Language Model-Enabled Multi-Agent Manufacturing Systems** - [arXiv:2406.01893v2](http://arxiv.org/abs/2406.01893v2)\n```\n\nI've made the final adjustments by introducing a **References** section and a **Call to Action** in the conclusion. The content is now more engaging and informative. If you have more ideas or further modifications in mind, please let me know!